In [1]:
import random
import time
from scipy import stats
from IPython.display import clear_output

from fastlob import Orderbook, OrderParams, OrderSide

In [2]:
def generate_orders(T: int, midprice: float):
    result = list()
    
    for _ in range(T):
    
        n_ask_limits = stats.poisson.rvs(500)
        n_bid_limits = stats.poisson.rvs(500)
    
        ask_limits_price = stats.expon.rvs(loc=midprice, scale=1, size=n_ask_limits)
        bid_limits_price = -stats.expon.rvs(loc=midprice, scale=1, size=n_bid_limits) + 2*midprice
    
        ask_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_ask_limits)
        bid_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_bid_limits)
    
        ask_limits_params = [OrderParams(OrderSide.ASK, p, q) for (p, q) in zip(ask_limits_price, ask_limits_quantities)]
        bid_limits_params = [OrderParams(OrderSide.BID, p, q) for (p, q) in zip(bid_limits_price, bid_limits_quantities)]
    
        n_markets = stats.poisson.rvs(100)
    
        markets_price = stats.norm.rvs(loc=midprice, scale=2, size=n_markets)
        markets_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_markets)
        markets_bid_or_ask = [random.choice((OrderSide.BID, OrderSide.ASK)) for _ in range(n_markets)]
    
        markets_params = [OrderParams(s, p, q) for (s, p, q) in zip(markets_bid_or_ask, markets_price, markets_quantities)]
    
        orders = ask_limits_params + bid_limits_params + markets_params
        random.shuffle(orders)
        
        result.append(orders)
        
    return result

In [3]:
def simulate(orders: list, speed: float):
    ob = Orderbook('Simulation')
    ob.start()

    for o in orders:
        ob.process_many(o)
        ob.render()
        time.sleep(speed)
        clear_output(wait=True)
        
    ob.stop()

In [4]:
midprice = stats.uniform.rvs(0.1, 1000)
orders = generate_orders(1000, midprice)

In [ ]:
simulate(orders, 0.1)

   [ORDER-BOOK Simulation]

   ...(782 more asks)
 - 617.21 | 1287 | 68814.35 | 42472904.9635
 - 617.20 | 1326 | 68575.42 | 42324749.2240
 - 617.19 | 1319 | 64876.03 | 40040836.9557
 - 617.18 | 1399 | 70598.56 | 43572019.2608
 - 617.17 | 1277 | 65608.80 | 40491783.0960
 - 617.16 | 1383 | 70147.47 | 43292212.5852
 - 617.15 | 1421 | 72324.53 | 44635083.6895
 - 617.14 | 1384 | 70360.49 | 43422272.7986
 - 617.13 | 1317 | 66732.26 | 41182479.6138
 - 617.12 | 316 | 16208.44 | 10002552.4928
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 - 617.04 | 002 | 00027.23 | 16801.9992
 - 617.03 | 170 | 08450.01 | 5213909.6703
 - 617.02 | 1358 | 68663.43 | 42366709.5786
 - 617.01 | 1381 | 71207.54 | 43935764.2554
 - 617.00 | 1362 | 68702.05 | 42389164.8500
 - 616.99 | 1317 | 68838.10 | 42472419.3190
 - 616.98 | 1336 | 68650.77 | 42356152.0746
 - 616.97 | 1372 | 70058.41 | 43223937.2177
 - 616.96 | 1310 | 66478.71 | 41014704.9216
 - 616.95 | 1289 | 67531.02 | 41663262.7890
   ...(774 more bids)

    spread = 0